In [1]:
import pandas as pd
import math
import time
import datetime as datetime
from dateutil import parser
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 25)

In [2]:
start_time = time.time()
today = datetime.datetime(2022, 6, 1)

In [3]:
df = pd.read_csv("https://projects.fivethirtyeight.com/polls-page/data/president_approval_polls.csv", parse_dates=['start_date', 'end_date', 'created_at'])
psters = pd.read_csv("C:/Users/horat/Python Project/Senate 2022/Test Forecast/pollster-ratings.csv")
output = pd.read_csv("C:/Users/horat/Python Project/Senate 2022/Test Forecast/biden-approval.csv")

In [4]:
df=df[(df['end_date'] < '2022-06-01')]

In [5]:
def assign_weight(n, days, g, v, internal): # n=sample size, days=days_old, v=grade, internal=pop_value
    # impute missing sample size
    if n=="":
        n=df['sample_size'].median # or 400 for district-level polls, 600 for state-level polls and 1,200 for national polls
    else:
        pass
    # weight by sample size
    if n > 3500:
        n_weight = 55.22
    else:
        n_weight = 5 / (1 / math.pow(n, .3)) # https://fivethirtyeight.com/features/polls-now-weighted-by-sample-size/
    # weight by recency
    aggr = 20
    recency = (90 / (1 + math.pow(1.5, (days - aggr) / (aggr / 10))) + 10) / 100
    # internal polls weight
    if internal == "":
        partisan_effect = 1
    else: 
        partisan_effect = 0.7
    w = n_weight * v * partisan_effect * recency * g
    return w

In [6]:
# pollster df to dictionary -> id: letter_grade, grade_value, house_bias
def pollster_id_grade_bias(df):
    return dict([(rating_id,[grade,grarde_value,house_bias ]) for rating_id, grade,grarde_value,house_bias in zip(df.pollster_rating_id, df.grade,df.grade_value,df.house_bias)])

In [7]:
def add_pollsters_info(df):
    pop_val = {'v' : 1, 'lv' : 1, 'rv' : 0.67, 'a' : 0.5, '' : 0.5} # create voter grades
    df['pop_value'] = df['population'].map(pop_val)
    for index, row in df.iterrows():
        try: # catch pollsters which don't exist in pollster_rating information
            df.at[index, "grade_symbol"]=pollster_id_grade_bias(psters)[row['pollster_rating_id']][0]
            df.at[index, "grade"]=pollster_id_grade_bias(psters)[row['pollster_rating_id']][1]
            df.at[index, "house_bias"]=pollster_id_grade_bias(psters)[row['pollster_rating_id']][2]
        except:
            df.at[index, "house_bias"] = 0
            df.at[index, "grade"] = 0.6
            df.at[index, "grade_symbol"] = ""
    return df

In [8]:
df=add_pollsters_info(df)
df

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,politician_id,politician,yes,no,alternate_answers,pop_value,grade_symbol,grade,house_bias
1205,79057,399,Rasmussen (Pulse Opin...,1753,Matt Palumbo's The Ma...,Rasmussen Reports,277,Rasmussen Reports,B,IVR/Online,NaN,2022-05-29,2022-05-31,NaN,NaN,NaN,156036,1500.0,lv,NaN,lv,True,2022-06-01 11:02:00,NaN,https://www.rasmussen...,NaN,False,NaN,42,Joe Biden,42.00,56.00,0.00,1.00,B-,0.93,-0.8942
1206,79055,568,YouGov,352,Economist,YouGov,391,YouGov,B+,Online,NaN,2022-05-28,2022-05-31,NaN,NaN,NaN,156016,1500.0,a,NaN,a,NaN,2022-06-01 09:45:00,NaN,https://today.yougov....,NaN,False,NaN,42,Joe Biden,41.00,49.00,9.00,0.50,B-,0.93,0.3595
1207,79055,568,YouGov,352,Economist,YouGov,391,YouGov,B+,Online,NaN,2022-05-28,2022-05-31,NaN,NaN,NaN,156021,1280.0,rv,NaN,rv,NaN,2022-06-01 09:57:00,NaN,https://docs.cdn.youg...,NaN,False,NaN,42,Joe Biden,43.00,51.00,6.00,0.67,B-,0.93,0.3595
1208,79070,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online,NaN,2022-05-25,2022-05-31,NaN,NaN,NaN,156097,45000.0,a,NaN,a,True,2022-06-02 08:31:00,NaN,https://morningconsul...,NaN,False,NaN,42,Joe Biden,40.00,53.00,8.00,0.50,B,1.00,0.2757
1209,80790,1193,SurveyMonkey,NaN,NaN,SurveyMonkey,324,SurveyMonkey,C,Online,NaN,2022-05-01,2022-05-31,NaN,NaN,NaN,162061,46495.0,a,NaN,a,NaN,2022-09-24 11:32:00,NaN,https://www.surveymon...,NaN,False,NaN,42,Joe Biden,41.00,57.00,0.00,0.50,C,0.75,0.3335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3239,74248,241,Ipsos,71,Reuters,Ipsos,154,Ipsos,B-,Online,NaN,2021-01-20,2021-01-21,NaN,NaN,NaN,139404,1115.0,a,NaN,a,NaN,2021-01-22 22:06:00,NaN,https://www.ipsos.com...,538.0,False,NaN,42,Joe Biden,55.47,31.59,12.94,0.50,B+,1.05,0.4461
3240,74248,241,Ipsos,71,Reuters,Ipsos,154,Ipsos,B-,Online,NaN,2021-01-20,2021-01-21,NaN,NaN,NaN,139405,950.0,rv,NaN,rv,NaN,2021-01-22 22:07:00,NaN,https://www.ipsos.com...,538.0,False,NaN,42,Joe Biden,57.00,32.00,11.00,0.67,B+,1.05,0.4461
3241,74327,568,YouGov,1213,Yahoo News,YouGov,391,YouGov,B+,Online,NaN,2021-01-20,2021-01-21,NaN,NaN,NaN,139570,1516.0,a,NaN,a,NaN,2021-02-02 15:39:00,NaN,https://docs.cdn.youg...,538.0,False,NaN,42,Joe Biden,45.00,28.00,26.00,0.50,B-,0.93,0.3595
3242,74247,399,Rasmussen (Pulse Opin...,1585,SLANTED,Rasmussen Reports,277,Rasmussen Reports,B,IVR/Online,NaN,2021-01-19,2021-01-21,NaN,NaN,NaN,139395,1500.0,lv,NaN,lv,True,2021-01-22 10:58:00,NaN,https://www.rasmussen...,538.0,False,NaN,42,Joe Biden,48.00,45.00,0.00,1.00,B-,0.93,-0.8942


In [9]:
# there are polls with the same poll_id overlap, only the more reliable sample type is chosen via pop_value
fp = pd.DataFrame()
fp = df.groupby("poll_id")["pop_value"].max().reset_index().merge(df, on=["poll_id", "pop_value"], how="inner")
# for d in poll_ids:
#     answers = df[df["poll_id"] == d]
#     best_grade = max(answers["pop_value"])
#     pb = answers[answers["pop_value"] == best_grade]
#     fp = fp.append(pb, ignore_index=True)

In [10]:
# organize df1 with date
fp.sort_values(by='end_date', ascending = False, inplace = True)
fp=fp.reset_index(drop=True)
# reorganize columns order to that of df
column_headers = list(df.columns.values)
fp=fp.loc[:,column_headers]
fp

,poll_id,pollster_id,pollster,sponsor_ids,sponsors,display_name,pollster_rating_id,pollster_rating_name,fte_grade,methodology,state,start_date,end_date,sponsor_candidate_id,sponsor_candidate,sponsor_candidate_party,question_id,sample_size,population,subpopulation,population_full,tracking,created_at,notes,url,source,internal,partisan,politician_id,politician,yes,no,alternate_answers,pop_value,grade_symbol,grade,house_bias
0,80790,1193,SurveyMonkey,NaN,NaN,SurveyMonkey,324,SurveyMonkey,C,Online,NaN,2022-05-01,2022-05-31,NaN,NaN,NaN,162077,39828.0,rv,NaN,rv,NaN,2022-09-24 11:41:00,NaN,https://www.surveymon...,NaN,False,NaN,42,Joe Biden,42.0,58.0,0.0,0.67,C,0.75,0.3335
1,79055,568,YouGov,352,Economist,YouGov,391,YouGov,B+,Online,NaN,2022-05-28,2022-05-31,NaN,NaN,NaN,156021,1280.0,rv,NaN,rv,NaN,2022-06-01 09:57:00,NaN,https://docs.cdn.youg...,NaN,False,NaN,42,Joe Biden,43.0,51.0,6.0,0.67,B-,0.93,0.3595
2,79057,399,Rasmussen (Pulse Opin...,1753,Matt Palumbo's The Ma...,Rasmussen Reports,277,Rasmussen Reports,B,IVR/Online,NaN,2022-05-29,2022-05-31,NaN,NaN,NaN,156036,1500.0,lv,NaN,lv,True,2022-06-01 11:02:00,NaN,https://www.rasmussen...,NaN,False,NaN,42,Joe Biden,42.0,56.0,0.0,1.00,B-,0.93,-0.8942
3,79070,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online,NaN,2022-05-25,2022-05-31,NaN,NaN,NaN,156097,45000.0,a,NaN,a,True,2022-06-02 08:31:00,NaN,https://morningconsul...,NaN,False,NaN,42,Joe Biden,40.0,53.0,8.0,0.50,B,1.00,0.2757
4,79069,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online,NaN,2022-05-24,2022-05-30,NaN,NaN,NaN,156096,45000.0,a,NaN,a,True,2022-06-02 08:31:00,NaN,https://morningconsul...,NaN,False,NaN,42,Joe Biden,40.0,53.0,0.0,0.50,B,1.00,0.2757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,74247,399,Rasmussen (Pulse Opin...,1585,SLANTED,Rasmussen Reports,277,Rasmussen Reports,B,IVR/Online,NaN,2021-01-19,2021-01-21,NaN,NaN,NaN,139395,1500.0,lv,NaN,lv,True,2021-01-22 10:58:00,NaN,https://www.rasmussen...,538.0,False,NaN,42,Joe Biden,48.0,45.0,0.0,1.00,B-,0.93,-0.8942
1815,74272,1189,Morning Consult,NaN,NaN,Morning Consult,218,Morning Consult,B-,Online,NaN,2021-01-19,2021-01-21,NaN,NaN,NaN,139491,15000.0,a,NaN,a,True,2021-01-28 19:54:00,NaN,https://morningconsul...,538.0,False,NaN,42,Joe Biden,50.0,28.0,0.0,0.50,B,1.00,0.2757
1816,74327,568,YouGov,1213,Yahoo News,YouGov,391,YouGov,B+,Online,NaN,2021-01-20,2021-01-21,NaN,NaN,NaN,139570,1516.0,a,NaN,a,NaN,2021-02-02 15:39:00,NaN,https://docs.cdn.youg...,538.0,False,NaN,42,Joe Biden,45.0,28.0,26.0,0.50,B-,0.93,0.3595
1817,74248,241,Ipsos,71,Reuters,Ipsos,154,Ipsos,B-,Online,NaN,2021-01-20,2021-01-21,NaN,NaN,NaN,139405,950.0,rv,NaN,rv,NaN,2021-01-22 22:07:00,NaN,https://www.ipsos.com...,538.0,False,NaN,42,Joe Biden,57.0,32.0,11.0,0.67,B+,1.05,0.4461


In [11]:
p_output = fp.loc[:, ["pollster", "pollster_rating_id", "grade", "end_date", "population", "sample_size", "yes",  "no"]]
p_output

,pollster,pollster_rating_id,grade,end_date,population,sample_size,yes,no
0,SurveyMonkey,324,0.75,2022-05-31,rv,39828.0,42.0,58.0
1,YouGov,391,0.93,2022-05-31,rv,1280.0,43.0,51.0
2,Rasmussen (Pulse Opin...,277,0.93,2022-05-31,lv,1500.0,42.0,56.0
3,Morning Consult,218,1.00,2022-05-31,a,45000.0,40.0,53.0
4,Morning Consult,218,1.00,2022-05-30,a,45000.0,40.0,53.0
...,...,...,...,...,...,...,...,...
1814,Rasmussen (Pulse Opin...,277,0.93,2021-01-21,lv,1500.0,48.0,45.0
1815,Morning Consult,218,1.00,2021-01-21,a,15000.0,50.0,28.0
1816,YouGov,391,0.93,2021-01-21,a,1516.0,45.0,28.0
1817,Ipsos,154,1.05,2021-01-21,rv,950.0,57.0,32.0


In [12]:
max_output = parser.parse(max(output["date"])) # the latest date in the file
max_output

datetime.datetime(2022, 5, 17, 0, 0)

In [13]:
if (today - max_output).days == 1:
    start_date=today
else:
    start_date=max_output
num_days = (today - start_date).days
num_days

15

In [14]:
output = output[output["date"] < start_date.strftime("%Y-%m-%d")]
output

,date,yes,no,margin,weight
0,2021-01-21,51.34,34.10,17.24,148.18
1,2021-01-22,53.30,35.61,17.69,151.91
2,2021-01-23,53.60,35.02,18.58,165.73
3,2021-01-24,54.30,35.61,18.68,198.75
4,2021-01-25,54.99,34.62,20.38,283.94
...,...,...,...,...,...
476,2022-05-12,42.02,53.17,-11.14,630.06
477,2022-05-13,42.02,52.87,-10.84,624.51
478,2022-05-14,42.03,52.84,-10.81,600.23
479,2022-05-15,42.04,52.81,-10.78,575.99


In [15]:
pavg = pd.DataFrame()

In [52]:
for z in range(num_days+1):
    sim_date=start_date+datetime.timedelta(days=z) # sim_date=2022-5-17 00:00:00
    op = fp[fp["end_date"] <= sim_date].copy() # op=all rows <= sim_date
    u_ids = op["pollster_id"].unique() # u_ids=list of unique pollster id -> array([ 241,  399, 1189,  568, 1562, 1360, 1697,  396, 1575, 1347,  294,..., 907])
    op['days_old'] = (sim_date - op['end_date']).dt.days # calculate the date difference between end_date and sim_date
    pfsp = pd.DataFrame() # declare a new dataframe pfsp, length should be len(u_ids)
    for e in u_ids: # latest poll of every pollster and store into pfsp
        psp = op[op["pollster_id"] == e] # get every polls with pollster==reuter / pollster_id==241
        newest = max(psp["end_date"]) # Timestamp('2022-05-17 00:00:00')
        dsp = psp[psp["end_date"] == newest] # get the row with newest end_date (reuter poll with 2022-05-17 date)
        pfsp = pd.concat([pfsp, dsp], ignore_index=True)

    for index, row in pfsp.iterrows():
        weight = assign_weight(row["sample_size"], row["days_old"],
                               row["grade"], row["pop_value"], "")
        pfsp.at[index, "weight"] = round(weight, 3)
        pfsp.at[index, "yes_weight"] = round(weight * row["yes"], 3)
        pfsp.at[index, "no_weight"] = round(weight * row["no"], 3)
    avgs = pd.DataFrame({'date': [sim_date.strftime("%Y-%m-%d")],
                         'yes': [round(pfsp["yes_weight"].sum() / pfsp["weight"].sum(), 2)],
                         'no': [round(pfsp["no_weight"].sum() / pfsp["weight"].sum(), 2)],
                         'margin': [
                             round((pfsp["yes_weight"].sum() - pfsp["no_weight"].sum()) / pfsp["weight"].sum(), 2)],
                         'weight': round(pfsp["weight"].sum(), 2)})
    output = pd.concat([output, avgs], ignore_index=True)
    #print(sim_date, ":", avgs["margin"][0])

2022-05-17 00:00:00 : -10.95
2022-05-18 00:00:00 : -11.19
2022-05-19 00:00:00 : -11.61
2022-05-20 00:00:00 : -12.1
2022-05-21 00:00:00 : -12.16
2022-05-22 00:00:00 : -12.07
2022-05-23 00:00:00 : -12.04
2022-05-24 00:00:00 : -11.97
2022-05-25 00:00:00 : -12.23
2022-05-26 00:00:00 : -12.26
2022-05-27 00:00:00 : -12.1
2022-05-28 00:00:00 : -11.92
2022-05-29 00:00:00 : -12.11
2022-05-30 00:00:00 : -12.1
2022-05-31 00:00:00 : -12.37
2022-06-01 00:00:00 : -12.35


In [53]:
output.to_csv('C:/Users/horat/Python Project/Senate 2022/Test Forecast/biden-approval.csv', index=False, header=True)
p_output.to_csv('C:/Users/horat/Python Project/Senate 2022/Test Forecast/approval-polls.csv', index=False, header=True)